# Creating and Valuing a CDS Contract

In [1]:
# This program is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; 
# without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  
# See the license for more details.

Creates a CDS contract and does a valuation and calculates risk measures

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from financepy.finutils.FinDate import FinDate
from financepy.finutils.FinDayCount import FinDayCountTypes
from financepy.finutils.FinFrequency import FinFrequencyTypes
from financepy.finutils.FinMath import ONE_MILLION
from financepy.products.libor.FinLiborDeposit import FinLiborDeposit
from financepy.products.libor.FinLiborSwap import FinLiborSwap
from financepy.market.curves.FinLiborCurve import FinLiborCurve
from financepy.products.credit.FinCDS import FinCDS
from financepy.market.curves.FinCDSCurve import FinCDSCurve

## Creating a CDS Contract

In [4]:
tradeDate = FinDate(2019, 8, 9)
effectiveDate = tradeDate
maturityDate = FinDate(2029, 6, 20)
cdsCoupon = 0.0150
notional = ONE_MILLION
longProtection = False
tradeDate = FinDate(2019, 8, 9)

In [5]:
cdsContract = FinCDS(effectiveDate, maturityDate, cdsCoupon, notional, longProtection)

## Build Libor Curve

In [6]:
settlementDate = effectiveDate

In [7]:
dcType = FinDayCountTypes.ACT_360
depo1 = FinLiborDeposit(settlementDate, "1M", 0.022009, dcType)
depo2 = FinLiborDeposit(settlementDate, "2M", 0.022138, dcType)
depo3 = FinLiborDeposit(settlementDate, "3M", 0.021810, dcType)
depo4 = FinLiborDeposit(settlementDate, "6M", 0.020503, dcType)
depo5 = FinLiborDeposit(settlementDate, "12M", 0.019930, dcType)
depos = [depo1,depo2,depo3,depo4,depo5]

dcType = FinDayCountTypes.THIRTY_E_360_ISDA
fixedFreq = FinFrequencyTypes.SEMI_ANNUAL
swap1 = FinLiborSwap(settlementDate,"2Y",0.015910,fixedFreq,dcType)
swap2 = FinLiborSwap(settlementDate,"3Y",0.014990,fixedFreq,dcType)
swap3 = FinLiborSwap(settlementDate,"4Y",0.014725,fixedFreq,dcType)
swap4 = FinLiborSwap(settlementDate,"5Y",0.014640,fixedFreq,dcType)
swap5 = FinLiborSwap(settlementDate,"6Y",0.014800,fixedFreq,dcType)
swap6 = FinLiborSwap(settlementDate,"7Y",0.014995,fixedFreq,dcType)
swap7 = FinLiborSwap(settlementDate,"8Y",0.015180,fixedFreq,dcType)
swap8 = FinLiborSwap(settlementDate,"9Y",0.015610,fixedFreq,dcType)
swap9 = FinLiborSwap(settlementDate,"10Y",0.0159880,fixedFreq,dcType)
swap10 = FinLiborSwap(settlementDate,"12Y",0.016430,fixedFreq,dcType)
swaps = [swap1,swap2,swap3,swap4,swap5,swap6,swap7,swap8,swap9,swap10]

liborCurve = FinLiborCurve("USD_LIBOR", settlementDate, depos, [], swaps)

# Build a CDS Curve

In [8]:
cds1 = FinCDS(settlementDate, "1Y", 0.0200)
cds2 = FinCDS(settlementDate, "2Y", 0.0220)
cds3 = FinCDS(settlementDate, "3Y", 0.0250)
cds4 = FinCDS(settlementDate, "4Y", 0.0275)
cds5 = FinCDS(settlementDate, "5Y", 0.0290)
cds6 = FinCDS(settlementDate, "7Y", 0.0300)
cds7 = FinCDS(settlementDate, "10Y", 0.0310)
cds8 = FinCDS(settlementDate, "15Y", 0.0315)

In [9]:
cdss = [cds1, cds2, cds3, cds4, cds5, cds6, cds7, cds8]

In [10]:
recoveryRate = 0.40

In [11]:
issuerCurve = FinCDSCurve(settlementDate, cdss, liborCurve, recoveryRate)

In [12]:
print(issuerCurve)

TIME: SURVIVAL_PROBABILITY
0.0: 1.0
1.0020753363523363: 0.9666739727270788
2.0014127619496116: 0.9281284288276282
3.0007501875468865: 0.8800174944969157
4.000087613144162: 0.8281097157452817
5.002162949496498: 0.7790286813571315
7.000837800691048: 0.6964700478556446
10.001587988237935: 0.5852215860696448
15.001013026979372: 0.4420049332411179



# Valuation

In [13]:
spd = cdsContract.parSpread(settlementDate, issuerCurve, recoveryRate) * 10000.0
print("FAIR CDS SPREAD %10.5f bp"% spd)

FAIR CDS SPREAD  309.69246 bp


In [14]:
v = cdsContract.value(settlementDate, issuerCurve, recoveryRate)

In [15]:
full_pv = v['full_pv'] 
clean_pv = v['clean_pv']

In [16]:
print("FULL VALUE  %12.2f"% full_pv)
print("CLEAN VALUE %12.2f"% clean_pv)

FULL VALUE    -115921.40
CLEAN VALUE   -118004.73


In [17]:
cleanp = cdsContract.cleanPrice(settlementDate, issuerCurve, recoveryRate)
print("CLEAN PRICE %12.6f"% cleanp)

CLEAN PRICE    88.199845


In [18]:
accruedDays = cdsContract.accruedDays()
print("ACCRUED_DAYS", accruedDays)

ACCRUED_DAYS 50


In [19]:
accruedInterest = cdsContract.accruedInterest()
print("ACCRUED_COUPON", accruedInterest)

ACCRUED_COUPON 2083.3333333333335


In [20]:
protPV = cdsContract.protectionLegPV(settlementDate, issuerCurve, recoveryRate)
print("PROTECTION_PV", protPV)

PROTECTION_PV 228847.21568010873


In [21]:
premPV = cdsContract.premiumLegPV(settlementDate, issuerCurve, recoveryRate)
print("PREMIUM_PV", premPV)

PREMIUM_PV 112925.81922790298


In [22]:
rpv01 = cdsContract.riskyPV01(settlementDate, issuerCurve)
print("FULL_RPV01", rpv01['full_rpv01'])
print("CLEAN_RPV01", rpv01['clean_rpv01'])

FULL_RPV01 7.5283879485268645
CLEAN_RPV01 7.389499059637975


In [23]:
cdsContract.riskyPV01(settlementDate, issuerCurve)

{'full_rpv01': 7.5283879485268645, 'clean_rpv01': 7.389499059637975}

## Risk Measures

In [24]:
creditDV01 = cdsContract.creditDV01(settlementDate, issuerCurve, recoveryRate)

In [25]:
creditDV01

-650.1710235716164

Copyright (c) 2020 Dominic O'Kane